# AWS Redshift

In this notebook we are going to load the StackOverFlow developer survey 2017 data into reshift cluster and query the data. The function getpass("password") will capture a password for the database. 

In [ ]:
################################### SET THE FOLLOWING PARAMETERS ###################################################
#Set the AWS Region
region = 'us-west-2'

#Set the AWS Access ID (Given to you buy the DSA staff)
access_id = '<>' 

#Set the AWS Access Key (Given to you buy the DSA staff)
access_key = '<>' 

In [ ]:
import boto3
import random
import time
import json
import psycopg2
from getpass import getpass
from pandas import read_sql
import datetime

redshift_client = boto3.client('redshift',region_name=region, 
                   aws_access_key_id = access_id, 
                   aws_secret_access_key = access_key)

# Give a password to your redshift cluster
pwd = '<>' # Choose a password. It must be at least 8 characters long, must contain at least 1 decimal digit, and 
         #must contain at least 1 upper case character

Set the names of the security group for the cluster, names of the cluster and database itself. 

In [ ]:
Sec_group_name= "climate_sec_group"
cluster_name="climate"
database_name="climatecitydata"

Create an AWS EC2 client object to create a security group for the redshift cluster. We are going to deploy the cluster in us-west-1 region. 

In [ ]:
ec2_client = boto3.client('ec2',region_name=region, 
                   aws_access_key_id = access_id, 
                   aws_secret_access_key = access_key)


Security group named "redshift_Sec_group" is created.

In [ ]:
sg = ec2_client.create_security_group(
    Description='security group for redhift cluster',
    GroupName=Sec_group_name
)
Sec_group=sg["GroupId"]

Edit the security group inbound rules to allow all TCP/IP traffic on port number 5439. 

In [ ]:
try:
    sec_rule="ALL TCP"
    data = ec2_client.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': 5439,
             
             'ToPort': 5439,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        ],)
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")
#     print(data)


### Create a keypair

Create a keypair for the EC2 instance. We first generate a name to create a key with that name and also store the key in a file. ec2.create_key_pair() will create a keypair. System command echo is used to write the contents of keypair generated to a file created with same name as keypair. 

You have to modify the file permissions to provide readonly access. If the file is open, system will throw an error. Do chmod(file, 0o400) 

In [ ]:
import time 
import os
import getpass
from subprocess import call

#Set the username from system
system_user_name=getpass.getuser()

ec2_pem_file=time.strftime("EC2-%d%m%Y%H%M%S-"+system_user_name)
ec2_key=ec2_client.create_key_pair(KeyName=ec2_pem_file)

#Don't do this unless you have a good reason
#print(emr_key['KeyMaterial'])

os.system("echo \""+ec2_key['KeyMaterial']+"\" > "+ec2_pem_file+".pem")
os.chmod(ec2_pem_file+".pem",0o400)

print("KeyName         : "+ec2_key['KeyName']+"\nKey Fingerprint : "+ec2_key['KeyFingerprint'])

Below cell will deploy a redshift cluster. A default database named "sof_survey" is created during the cluster is deployed. The parameter "NumberOfNodes" will tell how many slave nodes the cluster should have. The security group created above is used. At the end of the session we will delete the security group. 

In [ ]:
response = redshift_client.create_cluster(
    DBName=database_name,            # Optional. A default database named dev is created for the cluster. Optionally, 
                                     # specify a custom database name (e.g. mydb) to create an additional database.
    
    ClusterIdentifier=cluster_name,  # Unique key that identifies a cluster. It is stored as a lowercase string. 
    ClusterType='multi-node',        # single-node is other option
    NodeType='dc1.large',            # other options are dc1.8xlarge ds2.xlarge ds2.8xlarge ds1.xlarge ds1.8xlarge
    MasterUsername='your_pawprint',  # Add your pawprint
    MasterUserPassword=pwd,
#     ClusterSubnetGroupName='default',
    VpcSecurityGroupIds=[
        Sec_group,
    ],
    ClusterParameterGroupName='default.redshift-1.0',  # Parameter group to associate with this cluster.  
    Port=5439,
    AllowVersionUpgrade=True,
    NumberOfNodes=2,   # Compute nodes store your data and execute your queries. In addition to your compute nodes, a leader 
                       # node will be added to your cluster, free of charge. The leader node is the access point for 
                       # ODBC/JDBC and generates the query plans executed on the compute nodes.
    
    PubliclyAccessible=True, # If true, cluster to be accessible from the public internet. If No, then its accessible only 
                             # from within the private VPC network
    EnhancedVpcRouting=False
)

Below poll function keeps checking the status of cluster. Once it is in reday state the poll function breaks out of the loop indicating the cluster is available for use.

In [ ]:
def poll_until_completed(client, cluster_id):
    delay = 2
    while True:
        cluster = client.describe_clusters(ClusterIdentifier=cluster_id)
#         for tag in cluster:
#             print(tag)
#         print(cluster)
#         print(cluster['Clusters'][0]['ClusterIdentifier'])
        status = cluster['Clusters'][0]['ClusterStatus']
#         message = cluster.get('Message', '')
        now = str(datetime.datetime.now().time())
        print("cluster %s is %s at %s" % (cluster_id, status, now))
        if status in ['available', 'final-snapshot']:
            break

        # exponential backoff with jitter
        delay *= random.uniform(1.1, 2.0)
        time.sleep(delay)

In [ ]:
poll_until_completed(redshift_client, cluster_id=cluster_name)  # Can't use the cluster until it is available

In [ ]:
# Run the cell if you want to see the complete details of cluster. 

# redshift_client.describe_clusters()

To connect to the cluster we need its endpoint. Below cell prints the end point, the default port where the cluster is listening for input requests and the database available in the cluster. 

In [ ]:
cluster_end_point = ''
for cluster in redshift_client.describe_clusters()["Clusters"]:
    print("Cluster endpoint:",str(cluster["Endpoint"]["Address"])+"\n"+"Port:",str(cluster["Endpoint"]["Port"])+"\n"+"Database:",str(cluster["DBName"]))
    cluster_end_point = str(cluster["Endpoint"]["Address"])

Below code cell prints the public and private addresses of the nodes in cluster. 

In [ ]:
for cluster in redshift_client.describe_clusters()["Clusters"]:
    for ClusterNode in cluster["ClusterNodes"]:
        if cluster_name in cluster["Endpoint"]["Address"]:
            print(ClusterNode)

Connection string below is used to connect to "sof_survey" database in "stackoverflow" using port 5439.  

In [ ]:
conn_string = { 'dbname': database_name, 
           'user':'your_pawprint', #Add your pawprint
           'pwd':pwd,
           'host':cluster_end_point,
           'port':'5439'
         }

In [ ]:
def create_conn(config):
    try:
        con=psycopg2.connect(dbname=config['dbname'], host=config['host'], 
                              port=config['port'], user=config['user'], 
                              password=config['pwd'])
        return con
    except Exception as err:
        print(err)

In [ ]:
con = create_conn(config=conn_string)

We have established the connection to redshift cluster using psycopg library. 

In [ ]:
con

## Upload the data to S3 bucket.

In [ ]:
import boto3
s3 = boto3.resource('s3',aws_access_key_id = access_id, 
                   aws_secret_access_key = access_key)


bucket_name=time.strftime(system_user_name+"bucket%S")

s3.create_bucket(Bucket=bucket_name)

s3.Object(bucket_name, 'GlobalLandTemperaturesByCity.csv').put(Body=open('GlobalLandTemperaturesByCity.csv', 'rb'))

In [ ]:
bucket = s3.Bucket(bucket_name)
bucket.Acl().put(ACL='public-read')

In [ ]:
import csv, ast, psycopg2

import csv
f = open('GlobalLandTemperaturesByCity.csv', 'r', encoding='latin-1')
reader = csv.reader(f)
    
# f = open('globalterrorismdb.csv', 'r')
# reader = csv.reader(f)
print(type(reader))
# Below line of code, will assign empty lists to variables longest, headers and type_list. We will use these variables in cells 
# Below when determining the type and size of each column in the table. 
# longest: holds the column size 
# headers: holds the column headers
# type_list: holds the column types in the dataset

longest, headers, type_list = [], [], []


# import pandas as pd
# df = pd.read_csv("globalterrorismdb.csv",encoding='latin-1',low_memory=False)
# df.head()

In [ ]:
import datetime

In [ ]:
def dataType(val, current_type):
    try:
        # Evaluates numbers to an appropriate type, and strings an error
        t = ast.literal_eval(val)
    except ValueError:
        return 'varchar'
    except SyntaxError:
        return 'varchar'
    
    if type(t) in [int, float]:
        if (type(t) in [int]) and current_type not in ['float', 'varchar']:
           # Use smallest possible int type
            if (-32768 < t < 32767) and current_type not in ['int', 'bigint']:
                return 'smallint'
            elif (-2147483648 < t < 2147483647) and current_type not in ['bigint']:
                return 'int'
            else:
                return 'bigint'
        if type(t) is float and current_type not in ['varchar']:
            return 'decimal'
    elif type(t) in [datetime]:
        return 'date'
    else:
        return 'varchar'

In [ ]:
# def dataType(val):
#     try:
#         # Evaluates numbers to an appropriate type, and strings an error
#         t = ast.literal_eval(val)
#     except ValueError:
#         return 'varchar'
#     except SyntaxError:
#         return 'varchar'
    

# # check if the cell value is integer type. If yes, return integer. In else case, check if its float type. Return 'float' if yes 
# # or return 'varchar' as the data type of cell.
#     try:
#         if isinstance(t, int):
#             return "int"
#         elif isinstance(t, float):
#             return "float"
#         else:
#             return "varchar"
#     except Exception as e:
#         print(e)

In [ ]:
for row in reader:
    if len(headers) == 0:
        headers = row
        for col in row:
            longest.append(0)
            type_list.append('')
    else:
        for i in range(len(row)):
            # NA is the csv null value
            if row[i] == 'NA':
                pass
            else:
                var_type = dataType(row[i], type_list[i])
                type_list[i] = var_type
                if len(row[i]) > longest[i]:
                    longest[i] = len(row[i])
f.close()

In [ ]:
# for row in reader:
#     if len(headers) == 0:
#         headers = row
#         for col in row:
#             longest.append(0)
#             type_list.append('')
            
            
#     else:
# #         print(type_list)
# #         print(len(row))
#         for i in range(len(row)):
#             if row[i] == 'NA':
#                 pass
#             else:
#                 type_list[i] = dataType(row[i])
#                 if len(row[i]) > longest[i]:
#                     longest[i] = len(row[i])+5       
# f.close()

In [ ]:
statement = 'create table '+database_name+' ('

for i in range(len(headers)):
    if type_list[i] == 'varchar':
        statement = (statement + '{} varchar({}),').format(headers[i].lower(), str(longest[i]))
    else:
        statement = (statement + '{} {}' + ',').format(headers[i].lower(), type_list[i])

statement = statement[:-1] + ');'

In [ ]:
statement

In [ ]:
statement='create table dsaclimatecitydata (dt date,averagetemperature numeric(10,5),averagetemperatureuncertainty numeric(10,5),city varchar(25),country varchar(34),latitude varchar(6),longitude varchar(7));'

In [ ]:
# df = read_sql("delete global_ter_data;",con=con)

In [ ]:
cur = con.cursor()

In [ ]:
cur.execute(statement)
con.commit()

In [ ]:
# con.rollback()

In [ ]:
bucket_name="rsgt3bbucket05"
 s3 = boto3.resource('s3',region_name=region, 
                   aws_access_key_id = access_id, 
                   aws_secret_access_key = access_key)

bucket = s3.Bucket(bucket_name)
bucket.Acl().put(ACL='public-read')

In [ ]:
print("s3://"+bucket_name+"/GlobalLandTemperaturesByCity.csv")

In [ ]:
sql = """copy dsaclimatecitydata from 's3://"""+bucket_name+"""/GlobalLandTemperaturesByCity.csv'
    access_key_id 'your_access_key_id' 
    secret_access_key 'your_secret_access_key'
    region 'us-east-1'
    ignoreheader 1
    null as 'NA'
    removequotes
    delimiter ',';"""
cur.execute(sql)
con.commit()

In [ ]:
# df=read_sql("select *from stl_load_errors",con)
# df

In [ ]:
df = read_sql("select * from dsaclimatecitydata limit 10;",con=con)
df

Lets create a table and load the data into Redshift. We established connection to the cluster above. Use the connection object "con" to execute create table srtatement.  

In [ ]:
df = read_sql("select column_name, data_type, character_maximum_length \
from INFORMATION_SCHEMA.COLUMNS where table_name = 'dsaclimatecitydata';",con=con)
df

We will stage the data on S3 first before writing it to redshift cluster. 

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
df = read_sql("""select * from dsaclimatecitydata where city = 'Hyderabad' limit 5;""",con)
df

In [ ]:
statement = """create table dsaclimatesubsetdata1 SORTKEY (dt) as select dt, averagetemperature, DATE_PART(month,dt) as month, 
                DATE_PART(year,dt) as year, city, country, latitude, longitude from dsaclimatecitydata;"""

In [ ]:
con.rollback()

In [ ]:
cur.execute(statement)
con.commit()

In [ ]:
# read_sql("""delete new_table""",con)

In [ ]:
df = read_sql("""select * from dsaclimatesubsetdata1 limit 5;""",con)
df

In [ ]:
df = read_sql("select column_name, data_type, character_maximum_length \
from INFORMATION_SCHEMA.COLUMNS where table_name = 'dsaclimatesubsetdata1';",con=con)
df

In [ ]:
df = read_sql("""select coalesce(median(averagetemperature),0) as median, year from dsaclimatesubsetdata1 
                group by year order by year;""",con)
df.head()

In [ ]:
import matplotlib.pyplot as plt

# Plot the data
plt.plot(df['year'],df['median'])

# Add a legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
con.rollback()

In [ ]:
df = read_sql("""select coalesce(median(averagetemperature),0) as median, year from dsaclimatesubsetdata1 
                 group by year having year >=1900 order by year;""",con)
df.head()

In [ ]:
import matplotlib.pyplot as plt

# Plot the data
plt.plot(df['year'],df['median'])

# Add a legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


trace=go.Scatter(
    x=df['year'],
    y=df['median'],
    mode='lines',
    )
data=[trace]

py.iplot(data, filename='line-mode')

In [ ]:
con.rollback()

In [ ]:
# li=['United States','China','India','Japan','Germany','United Kingdom']

df = read_sql("""select coalesce(median(averagetemperature),0) as median, year, country from dsaclimatesubsetdata1
                 where country in ('United States','China','India','Japan','Germany','United Kingdom') 
                 group by year,country having year >=1950 order by year;""",con)
df.head()

In [ ]:
abc=df.pivot('year','country','median')
f,ax=plt.subplots(figsize=(20,10))
abc.plot(ax=ax)

In [ ]:
con.rollback()

In [ ]:
df = read_sql("""select coalesce(median(averagetemperature),0) as median, year, country from dsaclimatesubsetdata1
                 where country in ('United States','China','India','Japan','Germany','United Kingdom') 
                 group by year,country order by year DESC limit 10;""",con)
df.head()

In [ ]:
df = read_sql("""select coalesce(max(averagetemperature),0) as max_temp, 
              country from dsaclimatesubsetdata1 group by country order by max_temp desc limit 20;""",con)
df.head()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")
# sns.despine(left=True, bottom=True)
bar_plot = sns.barplot(x=df["max_temp"], y=df["country"], palette="muted",orient="h")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df = read_sql("""select coalesce(max(averagetemperature),0) as max_temp, 
              city from dsaclimatesubsetdata1 group by city order by max_temp desc limit 20;""",con)
df.head()

In [ ]:
sns.set_style("darkgrid")
# sns.despine(left=True, bottom=True)
bar_plot = sns.barplot(x=df["max_temp"], y=df["city"], palette="muted",orient="h")
plt.xticks(rotation=45)
plt.show()

## Delete the cluster

In [ ]:
redshift_client

In [ ]:
# response = redshift_client.delete_cluster(
#     ClusterIdentifier='climate',
#     SkipFinalClusterSnapshot=True
# )

response = redshift_client.delete_cluster(
    ClusterIdentifier=cluster_name,
    SkipFinalClusterSnapshot=True
)